<a href="https://colab.research.google.com/github/SarathSabu/Python-Notebooks/blob/main/DBSCAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For Google Colab integration
import os
from google.colab import drive
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import pairwise_distances

from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
# import data as dataframe
file_path = '/content/drive/MyDrive/Infor648/Data/credit_card.csv'
df = pd.read_csv(file_path)


# calling head() method
df.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.00,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.00,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.00,0,1,1200.0,678.334763,244.791237,0.000000,12
4,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.00,0,8,1800.0,1400.057770,2407.246035,0.000000,12


#Select variables of interest

In [ ]:
df_sub= df[['PURCHASES_FREQUENCY', 'BALANCE', 'PRC_FULL_PAYMENT', 'PURCHASES_INSTALLMENTS_FREQUENCY']]

#Normalization or standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
#data standardization
scaler = StandardScaler()
data_std = scaler.fit_transform(df_sub)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#data normalization
norm_scaler = MinMaxScaler()
data_norm = norm_scaler.fit_transform(df_sub)
##data_norm is the normalized data

In [ ]:
from sklearn.cluster import DBSCAN
model = DBSCAN(eps=0.1, min_samples=5)
model.fit(data_norm)
df_sub['cluster'] = model.fit_predict(data_norm)






<ipython-input-7-c7c27c54e0e8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['cluster'] = model.fit_predict(data_norm)


#Evaluation
#Low Outlier Ratio (< 5%)
#Moderate Outlier Ratio (5% - 20%)
#High Outlier Ratio (>20%)

###Which one is a good outlier ratio highly depends on your specific dataset and problem domain

In [ ]:
##you do not need to change anything here
from scipy.spatial.distance import cdist

def calculate_dbscan_metrics(data_norm, eps, min_samples):
    # Fit DBSCAN model
    model = DBSCAN(eps=eps, min_samples=min_samples)
    labels = model.fit_predict(data_norm)

    # Number of clusters (excluding outlier)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    # Outlier ratio
    outlier_ratio = np.sum(labels == -1) / len(labels)

    # Calculate SSE (Sum of Squared Errors) within clusters
    sse = 0
    cluster_centers = {}
    for cluster_label in set(labels):
        if cluster_label == -1:
            continue
        cluster_points = data_norm[labels == cluster_label]
        cluster_center = np.mean(cluster_points, axis=0)
        cluster_centers[cluster_label] = cluster_center
        sse += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1) ** 2)

    # Calculate SSB (Sum of Squared Between-cluster)
    overall_mean = np.mean(data_norm, axis=0)
    ssb = 0
    for cluster_label, center in cluster_centers.items():
        cluster_size = np.sum(labels == cluster_label)
        ssb += cluster_size * np.sum((center - overall_mean) ** 2)

    # Total SSE + SSB
    sse_ssb_sum = sse + ssb

    # Calculate Silhouette Score (ignoring outliers)
    if n_clusters > 1:
        silhouette_avg = silhouette_score(data_norm[labels != -1], labels[labels != -1])
    else:
        silhouette_avg = np.nan  # Silhouette score is undefined for 1 or no clusters

    # Create DataFrame to display the results
    results = {
        'eps': [eps],
        'min_samples': [min_samples],
        'n_clusters': [n_clusters],
        'outlier_ratio': [outlier_ratio],
        'SSE': [sse],
        'SSB': [ssb],
        'SSE + SSB': [sse_ssb_sum],
        'Silhouette Score': [silhouette_avg]
    }

    df_results = pd.DataFrame(results)
    return df_results

#Consider other possible eps_value and min_sample_value

In [ ]:
# Define parameter ranges
# Create an array of eps values starting from 0.1 to 1.0 (exclusive), with a step size of 0.1.
# This means eps will take values like 0.1, 0.2, 0.3, ..., up to 0.9.
eps_values = np.arange(0.1, 1.0, 0.1) # change eps_value here
min_samples_values = np.arange(3, 10) # change min_sample here

# List to store the results
results = []

# Loop through each combination of eps and min_samples
for eps in eps_values:
    for min_samples in min_samples_values:
        # Apply DBSCAN and calculate metrics
        df_metrics = calculate_dbscan_metrics(data_norm, eps, min_samples)
        results.append(df_metrics)

# Concatenate all results
results_df = pd.concat(results, ignore_index=True)
pd.options.display.float_format = "{:.2f}".format

# Sort by SSE + SSB
results_df_sorted = results_df.sort_values(by='SSE + SSB', ascending=True)
display(results_df_sorted)

# Sort by Silhouette Score
results_df_sorted = results_df.sort_values(by='Silhouette Score', ascending=False)
display(results_df_sorted)


,eps,min_samples,n_clusters,outlier_ratio,SSE,SSB,SSE + SSB,Silhouette Score
6,0.10,9,5,0.03,3429.04,86.33,3515.38,-0.09
5,0.10,8,4,0.02,3512.89,15.87,3528.76,-0.16
4,0.10,7,5,0.02,3523.78,19.86,3543.64,-0.10
3,0.10,6,4,0.02,3548.81,10.34,3559.15,-0.21
2,0.10,5,3,0.01,3562.90,5.11,3568.01,-0.19
...,...,...,...,...,...,...,...,...
58,0.90,5,1,0.00,3621.38,0.00,3621.38,NaN
59,0.90,6,1,0.00,3621.38,0.00,3621.38,NaN
60,0.90,7,1,0.00,3621.38,0.00,3621.38,NaN
61,0.90,8,1,0.00,3621.38,0.00,3621.38,NaN


,eps,min_samples,n_clusters,outlier_ratio,SSE,SSB,SSE + SSB,Silhouette Score
6,0.10,9,5,0.03,3429.04,86.33,3515.38,-0.09
4,0.10,7,5,0.02,3523.78,19.86,3543.64,-0.10
0,0.10,3,5,0.01,3576.62,8.45,3585.07,-0.14
5,0.10,8,4,0.02,3512.89,15.87,3528.76,-0.16
2,0.10,5,3,0.01,3562.90,5.11,3568.01,-0.19
...,...,...,...,...,...,...,...,...
58,0.90,5,1,0.00,3621.38,0.00,3621.38,NaN
59,0.90,6,1,0.00,3621.38,0.00,3621.38,NaN
60,0.90,7,1,0.00,3621.38,0.00,3621.38,NaN
61,0.90,8,1,0.00,3621.38,0.00,3621.38,NaN


#Rerun the model with better eps and min_samples

In [ ]:
eps = 0.1
min_samples = 9
df_metrics = calculate_dbscan_metrics(data_norm, eps, min_samples)

# Display the metrics DataFrame
display(df_metrics)

,eps,min_samples,n_clusters,outlier_ratio,SSE,SSB,SSE + SSB,Silhouette Score
0,0.10,9,5,0.03,3429.04,86.33,3515.38,-0.09


In [ ]:
eps = 0.1
min_samples = 8
df_metrics = calculate_dbscan_metrics(data_norm, eps, min_samples)

# Display the metrics DataFrame
display(df_metrics)

,eps,min_samples,n_clusters,outlier_ratio,SSE,SSB,SSE + SSB,Silhouette Score
0,0.10,8,4,0.02,3512.89,15.87,3528.76,-0.16


In [ ]:
###You do not need to change anything here
def generate_cluster_profile_dbscan(data_norm, eps, min_samples, df_original):
    # Apply DBSCAN with customizable eps and min_samples
    dbscan_stats = DBSCAN(eps=eps, min_samples=min_samples)
    df_sub = df_original.copy()
    df_sub['cluster'] = dbscan_stats.fit_predict(data_norm)

    # Create a copy of the DataFrame for calculations
    df_calculate = df_sub.copy()
    df_calculate['cluster_result'] = 'Cluster ' + (df_calculate['cluster']).astype(str)

    # Exclude the 'cluster' column for mean calculation
    df_mean_calculation = df_calculate.drop(columns=['cluster'])
    df_mean_feature = df_mean_calculation.drop(columns=['cluster_result'])

    # Calculate Overall Mean for All Features in df_mean_feature
    overall_means = df_mean_feature.mean().to_frame().T
    overall_means.index = ['Overall']


    df_cluster_summary = df_mean_calculation.groupby('cluster_result').mean()


    df_profile = pd.concat([df_cluster_summary, overall_means], axis=0)

    # Calculate the count of items in each cluster
    cluster_counts = df_calculate['cluster_result'].value_counts()

    # Calculate the percentage of items in each cluster
    cluster_percentages = (cluster_counts / cluster_counts.sum()) * 100

    # Create a DataFrame with counts and percentages
    df_count_percentage = pd.DataFrame({
        'Count': cluster_counts,
        'Percentage': cluster_percentages
    })

    # Add a row for "Overall"
    df_count_percentage.loc['Overall'] = [len(df_calculate), 100.0]
    df_profile = pd.concat([df_profile, df_count_percentage], axis=1)
    df_overall = df_profile.loc['Overall']
    df_profile = df_profile.drop(index='Overall')

    # Sort the clusters by the Count column
    df_profile = df_profile.sort_values(by='Count', ascending=False)

    # Rename 'Cluster -1' to 'Outliers' in the final DataFrame
    df_profile.index = df_profile.index.str.replace('Cluster -1', 'Outliers')

    # Append the "Overall" row back to the sorted DataFrame
    df_profile = pd.concat([df_profile, df_overall.to_frame().T])

    # Format the profile DataFrame
    df_profile = df_profile.style.format({
        "Count": "{:.0f}",
        **{col: "{:.2f}" for col in df_profile.columns if col != "Count"}  # Two decimal places for all other columns
    }).background_gradient(cmap='Purples')  ###### Change color here to 'Blues', 'Purples', 'Oranges', etc.

    return df_profile



In [ ]:
# data_norm is our normalized data, eps = 0.1 amd min_samples = 8 df_sub is our selected subsect dataset with features of interest
profile_1 = generate_cluster_profile_dbscan(data_norm, eps=0.1, min_samples=9, df_original=df_sub)
display(profile_1)
print()


profile_2 = generate_cluster_profile_dbscan(data_norm, eps=0.1, min_samples=8, df_original=df_sub)
display(profile_2)


,PURCHASES_FREQUENCY,BALANCE,PRC_FULL_PAYMENT,PURCHASES_INSTALLMENTS_FREQUENCY,Count,Percentage
Cluster 0,0.49,1560.54,0.14,0.37,8273,95.80
Outliers,0.71,3548.03,0.38,0.33,253,2.93
Cluster 1,0.96,214.95,0.92,0.03,79,0.91
Cluster 3,0.54,135.50,0.71,0.00,16,0.19
Cluster 2,0.20,78.77,0.51,0.16,9,0.10
Cluster 4,0.46,54.37,0.51,0.01,6,0.07
Overall,0.50,1601.22,0.16,0.37,8636,100.00


,PURCHASES_FREQUENCY,BALANCE,PRC_FULL_PAYMENT,PURCHASES_INSTALLMENTS_FREQUENCY,Count,Percentage
Cluster 0,0.49,1545.93,0.15,0.37,8387,97.12
Outliers,0.69,4020.15,0.37,0.35,213,2.47
Cluster 1,0.56,162.35,0.70,0.00,19,0.22
Cluster 3,0.99,253.48,0.63,0.43,11,0.13
Cluster 2,0.46,54.37,0.51,0.01,6,0.07
Overall,0.50,1601.22,0.16,0.37,8636,100.00
